In [ ]:
%pip install autoflow-ai==0.0.1.dev3
%pip install dotenv
%pip install sqlalchemy

In [20]:
import logging
import os
import dotenv
from sqlalchemy import create_engine

logger = logging.getLogger(__name__)

dotenv.load_dotenv()

True

#### Create knowledge base

In [21]:
from autoflow import Autoflow
from autoflow.schema import IndexMethod
from autoflow.llms.chat_models import ChatModel
from autoflow.llms.embeddings import EmbeddingModel


db_engine = create_engine(os.getenv("DATABASE_URL"))
af = Autoflow(db_engine=db_engine)

chat_model = ChatModel("gpt-4o-mini")
embed_model = EmbeddingModel(model_name="text-embedding-3-small", dimensions=1536)

# Create Knowledge base
kb = af.create_knowledge_base(
    name="New KB",
    description="This is a knowledge base for testing",
    index_methods=[IndexMethod.VECTOR_SEARCH, IndexMethod.KNOWLEDGE_GRAPH],
    chat_model=chat_model,
    embedding_model=embed_model,
)
kb

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/autoflow/utils/sql_model_metaclass.py:206: SAWarning: This declarative base already contains a class with the same class name and module name as autoflow.models.chunk.Chunk, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/autoflow/utils/sql_model_metaclass.py:206: SAWarning: This declarative base already contains a class with the same class name and module name as autoflow.models.entity.Entity, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/autoflow/utils/sql_model_metaclass.py:206: SAWarning: This declarative base already contains a class with the same class name and module name as autoflow.models.relationship.Relationship

KnowledgeBase(id=UUID('4258ccab-0ab3-4c63-ae52-783d47790083'), name='New KB', index_methods=[<IndexMethod.VECTOR_SEARCH: 'VECTOR_SEARCH'>, <IndexMethod.KNOWLEDGE_GRAPH: 'KNOWLEDGE_GRAPH'>], description='This is a knowledge base for testing', chunking_config=GeneralChunkingConfig(mode=<ChunkingMode.GENERAL: 'general'>, chunk_size=1200, chunk_overlap=200, paragraph_separator='\n\n\n'), data_sources=[])

#### Import documents from files

In [22]:
import os
current_dir = os.path.dirname(os.path.abspath("__file__"))
current_dir

'/Users/liangzhiyuan/Projects/tidb.ai/core/tests'

In [23]:
from pathlib import Path

kb.import_documents_from_files(
    files=[
        Path(current_dir) / "fixtures" / "tidb-overview.md",
    ]
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
19:00:31 - LiteLLM:INFO: utils.py:1123 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
19:00:31 - LiteLLM:INFO: utils.py:2954 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
19:00:40 - LiteLLM:INFO: utils.py:1123 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
19:00:40 - LiteLLM:INFO: utils.py:2954 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
19:00:43 - LiteLLM:INFO: utils.py:1123 - Wrapper: Completed Call, calling success

[]

In [32]:
result = kb.search_documents(
    query="What is TiDB?",
    similarity_top_k=2,
)
[
    (c.score, c.chunk.text)
    for c in result.chunks
]

[(0.7382171054172685,
  'What is TiDB Self-Managed Key features\n<!-- Localization note for TiDB:\n- English: use distributed SQL, and start to emphasize HTAP\n- Chinese: can keep "NewSQL" and emphasize one-stop real-time HTAP ("一栈式实时 HTAP")\n- Japanese: use NewSQL because it is well-recognized\n-->\nTiDB (/\'taɪdiːbi:/, "Ti" stands for Titanium) is an open-source distributed SQL database that supports Hybrid Transactional and Analytical Processing (HTAP) workloads. It is MySQL compatible and features horizontal scalability, strong consistency, and high availability. The goal of TiDB is to provide users with a one-stop database solution that covers OLTP (Online Transactional Processing), OLAP (Online Analytical Processing), and HTAP services. TiDB is suitable for various use cases that require high availability and strong consistency with large-scale data.\nTiDB Self-Managed is a product option of TiDB, where users or organizations can deploy and manage TiDB on their own infrastructure

In [38]:
kg = kb.search_knowledge_graph(
    query="What is TiDB?",
)
[
    (r.rag_description)
    for r in kg.relationships
]

INFO:autoflow.storage.graph_store.tidb.tidb_graph_store:Debug - SQL Query: 
SELECT `relationships_4258ccab-0ab3-4c63-ae52-783d47790083`.id, `relationships_4258ccab-0ab3-4c63-ae52-783d47790083`.description, `relationships_4258ccab-0ab3-4c63-ae52-783d47790083`.description_vec, `relationships_4258ccab-0ab3-4c63-ae52-783d47790083`.source_entity_id, `relationships_4258ccab-0ab3-4c63-ae52-783d47790083`.target_entity_id, `relationships_4258ccab-0ab3-4c63-ae52-783d47790083`.meta, `relationships_4258ccab-0ab3-4c63-ae52-783d47790083`.weight, `relationships_4258ccab-0ab3-4c63-ae52-783d47790083`.chunk_id, `relationships_4258ccab-0ab3-4c63-ae52-783d47790083`.document_id, `relationships_4258ccab-0ab3-4c63-ae52-783d47790083`.created_at, `relationships_4258ccab-0ab3-4c63-ae52-783d47790083`.updated_at, %s - anon_1.embedding_distance AS similarity_score, `entities_4258ccab-0ab3-4c63-ae52-783d47790083_1`.id AS id_1, `entities_4258ccab-0ab3-4c63-ae52-783d47790083_1`.entity_type, `entities_4258ccab-0ab3-4c

['TiDB -> TiDB can be deployed in a Self-Managed model, providing users with control over their database setup. -> Self-Managed',
 'TiDB -> TiDB Self-Managed is a deployment option of TiDB that allows users to manage the database on their own infrastructure. -> TiDB Self-Managed',
 'TiDB -> TiDB utilizes TiKV as its row-based storage engine to support real-time data replication. -> TiKV',
 'TiDB -> TiDB employs TiFlash as its columnar storage engine to ensure consistent data storage and real-time replication from TiKV. -> TiFlash',
 'TiDB -> TiDB Operator facilitates the management of TiDB on Kubernetes, automating operational tasks. -> TiDB Operator',
 'TiDB -> TiDB Cloud is the fully-managed service that allows users to deploy and run TiDB clusters in the cloud. -> TiDB Cloud',
 'TiDB Self-Managed -> TiDB Self-Managed utilizes TiDB Architecture to structure its deployment and operations. -> TiDB Architecture',
 'TiDB Self-Managed -> TiDB Self-Managed relies on TiDB Storage for data m